In [ ]:
# Importing libraries and dataset
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from torch.utils.data import Dataset, DataLoader
from torch import nn
from keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))

In [ ]:
class inputdataset(Dataset):
  def __init__(self,x,y):
    # initialize the x and y of the dataset and convert them to tensors
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    # return a particular index (both x and y value) in the dataset
    return self.x[idx],self.y[idx]

  def __len__(self):
    # return the size of the dataset
    return self.length

In [ ]:
train_dataset = inputdataset(x_train,y_train)
test_dataset = inputdataset(x_test,y_test)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset,batch_size=100,shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset,batch_size=100,shuffle=True)

In [ ]:
class net(nn.Module):
  def __init__(self,input_size,output_size):     # defining the layers 
    super(net,self).__init__()
    self.l1 = nn.Linear(input_size,5)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(5,output_size)

  def forward(self,x):    # defining the forward pass which is invoked automatically when the class is called
    output = self.l1(x) 
    output = self.relu(output)
    output = self.l2(output)
    return output

In [ ]:
model = net(28*28,10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)

epochs = 2000

In [ ]:
cost_values = []
loss_values = []

for epoch in range(epochs):
    for i, (x_train, y_train) in enumerate(train_dataloader):
        
        y_pred = model(x_train)
        y_train = y_train.long()
        cost = criterion(y_pred, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    if epoch % 50 == 0:
        cost_values.append(cost)
    
    # loss_values.append(train_loss / len(train_dataloader))


In [ ]:
total_correct = 0
total_samples = 0

with torch.no_grad():
    for test_inputs, test_targets in test_dataloader:
        test_outputs = model(test_inputs)
        _, predicted_labels = torch.max(test_outputs, dim=1)
        total_correct += (predicted_labels == test_targets).sum().item()
        total_samples += test_targets.size(0)

accuracy = total_correct / total_samples
print("Test accuracy:", accuracy)